In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import os
from scipy import optimize, stats, interpolate
from scipy.integrate import quad
import argparse
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
import sys
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
import healpy as hp
from copy import copy, deepcopy
# sys.path.append('BatML/')
import logging, traceback
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [2]:
NITRATES_path = '/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES/NITRATES_PY3'

%cd {NITRATES_path}

/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES/NITRATES_PY3


In [3]:
# if the response directories are all put in the same directory
# and given as an env variable (like in this cell), then config.py will 
# be able to find them correctly as long as they keep the original names 
# (Ex: ray trace directotry is still called 'ray_traces_detapp_npy')
# These paths can also just be hardcoded into config.py
# most of these paths can also be given directly as an argument to 
# some analysis objects, like Source_Model_InOutFoV()
os.environ['NITRATES_RESP_DIR'] = '/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/'

In [4]:
from config import rt_dir
from ray_trace_funcs import RayTraces
from event2dpi_funcs import det2dpis, mask_detxy
from flux_models import Cutoff_Plaw_Flux, Plaw_Flux, get_eflux_from_model
from models import Source_Model_InOutFoV, Bkg_Model_wFlatA,\
                CompoundModel, Point_Source_Model_Binned_Rates, im_dist
from ray_trace_funcs import RayTraces
from sqlite_funcs import get_conn
from dbread_funcs import get_info_tab, get_twinds_tab
from hp_funcs import ang_sep
from coord_conv_funcs import theta_phi2imxy, imxy2theta_phi, convert_imxy2radec,\
                            convert_radec2thetaphi, convert_radec2imxy
from do_llh_inFoV4realtime2 import parse_bkg_csv
from LLH import LLH_webins
from minimizers import NLLH_ScipyMinimize_Wjacob

In [5]:
%matplotlib inline

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [6]:
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]
ebins0 = np.round(ebins0, decimals=1)[:-1]
ebins1 = np.append(ebins0[1:], [350.0])
nebins = len(ebins0)
print("Nebins: ", nebins)
print("ebins0: ", ebins0)
print("ebins1: ", ebins1)

# work_dir = '/storage/work/jjd330/local/bat_data/realtime_workdir/F646018360/'
work_dir = os.path.join(NITRATES_path, 'F646018360')
conn = get_conn(os.path.join(work_dir,'results.db'))
info_tab = get_info_tab(conn)
trigger_time = info_tab['trigtimeMET'][0]
print('trigger_time: ', trigger_time)

evfname = os.path.join(work_dir,'filter_evdata.fits')
ev_data = fits.open(evfname)[1].data
GTI_PNT = Table.read(evfname, hdu='GTI_POINTING')
GTI_SLEW = Table.read(evfname, hdu='GTI_SLEW')
dmask = fits.open(os.path.join(work_dir,'detmask.fits'))[0].data
attfile = fits.open(os.path.join(work_dir,'attitude.fits'))[1].data

att_ind = np.argmin(np.abs(attfile['TIME'] - trigger_time))
att_quat = attfile['QPARAM'][att_ind]
print('attitude quaternion: ', att_quat)

ndets = np.sum(dmask==0)
print("Ndets: ", np.sum(dmask==0))

t_end = trigger_time + 1e3
t_start = trigger_time - 1e3
mask_vals = mask_detxy(dmask, ev_data)
bl_dmask = (dmask==0.)

bl_ev = (ev_data['EVENT_FLAGS']<1)&\
        (ev_data['ENERGY']<=500.)&(ev_data['ENERGY']>=14.)&\
        (mask_vals==0.)&(ev_data['TIME']<=t_end)&\
        (ev_data['TIME']>=t_start)

print("Nevents: ",np.sum(bl_ev))
ev_data0 = ev_data[bl_ev]

Nebins:  9
ebins0:  [ 15.   24.   35.   48.   64.   84.  120.  171.5 245. ]
ebins1:  [ 24.   35.   48.   64.   84.  120.  171.5 245.  350. ]
trigger_time:  646018383.1787
attitude quaternion:  [-0.03597053  0.2345147  -0.64420835  0.72712074]
Ndets:  14932
Nevents:  1367885


In [7]:
ra, dec = 233.117, -26.213 
theta, phi = convert_radec2thetaphi(ra, dec, att_quat)
print(theta, phi)
imx, imy = convert_radec2imxy(ra, dec, att_quat)
print(imx, imy)

38.54132137017975 137.65241966813443
-0.5887551341212709 -0.5366203642198198


In [8]:
flux_params = {'A':1.0, 'gamma':0.5, 'Epeak':1e2}
flux_mod = Cutoff_Plaw_Flux(E0=100.0)

In [9]:
# use rt_dir from config.py or
# rt_dir = '/path/to/ray_traces_detapp_npy/'
rt_obj = RayTraces(rt_dir)

INFO:root:0.02
INFO:root:0.002


In [10]:
rt = rt_obj.get_intp_rt(imx, imy)
print(np.shape(rt))

DEBUG:root:opening file: fwd_ray_trace_-0.60000_-0.54000_-0.58000_-0.52000_.npy
DEBUG:root:nbytes_total=95789408.0
(173, 286)


In [11]:
%%time
# will by default use the resp directories from config.py
sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
                                rt_obj, use_deriv=True)
# or the paths can be given
# resp_tab_dname = '/path/to/resp_tabs_ebins/'
# hp_flor_resp_dname = '/path/to/hp_flor_resps/'
# comp_flor_resp_dname = '/path/to/comp_flor_resps/'
# sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
#                                 rt_obj, use_deriv=True,\
#                                 resp_tab_dname=resp_tab_dname,\
#                                 hp_flor_resp_dname=hp_flor_resp_dname,\
#                                 comp_flor_resp_dname=comp_flor_resp_dname)

CPU times: user 1.38 ms, sys: 908 µs, total: 2.29 ms
Wall time: 1.16 ms


In [12]:
%%time
sig_mod.set_theta_phi(theta, phi)

INFO:root:Making new response object
(0.0, 30.48, -14.117)
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=15)
           2	LOAD_FAST(arg=0, lineno=15)
           4	LOAD_CONST(arg=1, lineno=15)
           6	BINARY_SUBSCR(arg=None, lineno=15)
           8	STORE_FAST(arg=1, lineno=15)
          10	LOAD_FAST(arg=0, lineno=16)
          12	LOAD_CONST(arg=2, lineno=16)
          14	BINARY_SUBSCR(arg=None, lineno=16)
          16	STORE_FAST(arg=2, lineno=16)
          18	LOAD_FAST(arg=0, lineno=17)
          20	LOAD_CONST(arg=3, lineno=17)
          22	BINARY_SUBSCR(arg=None, lineno=17)
          24	STORE_FAST(arg=3, lineno=17)
          26	LOAD_FAST(arg=2, lineno=18)
          28	LOAD_FAST(arg=1, lineno=18)
          30	BINARY_SUBTRACT(arg=None, lineno=18)
          32	STORE_FAST(arg=4, lineno=18)
          34	LOAD_FAST(arg=3, lineno=19)
          36	LOAD_FAST(arg=1, lineno=19)
          38	BINARY_SUBTRACT(arg=None, lineno=19)
          40	STORE_FAST(arg=5, lineno

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x7fc150dde790>


DEBUG:numba.core.ssa:on stmt: wall_bnds = arg(0, name=wall_bnds)
DEBUG:numba.core.ssa:on stmt: $const4.1 = const(int, 0)
DEBUG:numba.core.ssa:on stmt: x0 = static_getitem(value=wall_bnds, index=0, index_var=$const4.1, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $const12.4 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: x1 = static_getitem(value=wall_bnds, index=1, index_var=$const12.4, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $const20.7 = const(int, 2)
DEBUG:numba.core.ssa:on stmt: x2 = static_getitem(value=wall_bnds, index=2, index_var=$const20.7, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: v01 = x1 - x0
DEBUG:numba.core.ssa:on stmt: v02 = x2 - x0
DEBUG:numba.core.ssa:on stmt: $42load_global.15 = global(normalize_vec: CPUDispatcher(<function normalize_vec at 0x7fc1b17b8820>))
DEBUG:numba.core.ssa:on stmt: $44load_global.16 = global(np: <module 'numpy' from '/Users/tparsota/opt/anaconda3/lib/python3.8/site-packages/numpy/__init__.p

DEBUG:numba.core.byteflow:dispatch pc=38, inst=LOAD_ATTR(arg=2, lineno=4451)
DEBUG:numba.core.byteflow:stack ['$b_36.0']
DEBUG:numba.core.byteflow:dispatch pc=40, inst=LOAD_CONST(arg=1, lineno=4451)
DEBUG:numba.core.byteflow:stack ['$38load_attr.1']
DEBUG:numba.core.byteflow:dispatch pc=42, inst=BINARY_SUBSCR(arg=None, lineno=4451)
DEBUG:numba.core.byteflow:stack ['$38load_attr.1', '$const40.2']
DEBUG:numba.core.byteflow:dispatch pc=44, inst=LOAD_CONST(arg=2, lineno=4451)
DEBUG:numba.core.byteflow:stack ['$42binary_subscr.3']
DEBUG:numba.core.byteflow:dispatch pc=46, inst=COMPARE_OP(arg=7, lineno=4451)
DEBUG:numba.core.byteflow:stack ['$42binary_subscr.3', '$const44.4']
DEBUG:numba.core.byteflow:dispatch pc=48, inst=POP_JUMP_IF_FALSE(arg=58, lineno=4451)
DEBUG:numba.core.byteflow:stack ['$46compare_op.5']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=50, stack=(), blockstack=(), npush=0), Edge(pc=58, stack=(), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([Sta

DEBUG:numba.core.byteflow:block_infos State(pc_initial=50 nstack_initial=0):
AdaptBlockInfo(insts=((50, {'res': '$50load_global.0'}), (52, {'res': '$const52.1'}), (54, {'func': '$50load_global.0', 'args': ['$const52.1'], 'res': '$54call_function.2'}), (56, {'exc': '$54call_function.2'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.byteflow:block_infos State(pc_initial=58 nstack_initial=0):
AdaptBlockInfo(insts=((58, {'res': '$a_58.0'}), (60, {'item': '$a_58.0', 'res': '$60load_attr.1'}), (62, {'res': '$const62.2'}), (64, {'index': '$const62.2', 'target': '$60load_attr.1', 'res': '$64binary_subscr.3'}), (66, {'res': '$const66.4'}), (68, {'lhs': '$64binary_subscr.3', 'rhs': '$const66.4', 'res': '$68compare_op.5'}), (70, {'pred': '$68compare_op.5'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={72: (), 86: ()})
DEBUG:numba.core.byteflow:block_infos State(pc_initial=72 nstack_initial=0):
AdaptBlockInfo(i

...

DEBUG:numba.core.byteflow:Used_phis: defaultdict(<class 'set'>, {State(pc_initial=0 nstack_initial=0): set()})
DEBUG:numba.core.byteflow:defmap: {}
DEBUG:numba.core.byteflow:phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$pha_bins0_22.0'}), (4, {'res': '$e0_24.1'}), (6, {'lhs': '$pha_bins0_22.0', 'rhs': '$e0_24.1', 'res': '$6compare_op.2'}), (8, {'res': '$pha_bins1_28.3'}), (10, {'res': '$e0_210.4'}), (12, {'lhs': '$pha_bins1_28.3', 'rhs': '$e0_210.4', 'res': '$12compare_op.5'}), (14, {'lhs': '$6compare_op.2', 'rhs': '$12compare_op.5', 'res': '$14binary_and.6'}), (16, {'retval': '$14binary_and.6', 'castval':

DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>,
            {'$phi16.0': {('$14get_iter.5',
                           State(pc_initial=0 nstack_initial=0)),
                          ('$phi16.0', State(pc_initial=16 nstack_initial=1))},
             '$phi18.0': {('$14get_iter.5',
                           State(pc_initial=0 nstack_initial=0))},
             '$phi18.1': {('$16for_iter.2',
                           State(pc_initial=16 nstack_initial=1))}})
DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>,
            {'$phi16.0': {('$14get_iter.5',
                           State(pc_initial=0 nstack_initial=0))},
             '$phi18.0': {('$14get_iter.5',
                           State(pc_initial=0 nstack_initial=0))},
             '$phi18.1': {('$16for_iter.2',
                           State(pc_initial=16 nstack_initial=1))}})
DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>,
            {'$phi16.0': {('$14get_ite

DEBUG:numba.core.ssa:SSA violators {'c'}
DEBUG:numba.core.ssa:Fix SSA violator on var c
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._FreshVarHandler object at 0x7fc15115d550>
DEBUG:numba.core.ssa:on stmt: arr = arg(0, name=arr)
DEBUG:numba.core.ssa:on stmt: c = freevar(zero: 0)
DEBUG:numba.core.ssa:first assign: c
DEBUG:numba.core.ssa:replaced with: c = freevar(zero: 0)
DEBUG:numba.core.ssa:on stmt: $6load_global.1 = global(np: <module 'numpy' from '/Users/tparsota/opt/anaconda3/lib/python3.8/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $8load_method.2 = getattr(value=$6load_global.1, attr=nditer)
DEBUG:numba.core.ssa:on stmt: $12call_method.4 = call $8load_method.2(arr, func=$8load_method.2, args=[Var(arr, arraymath.py:167)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $14get_iter.5 = getiter(value=$12call_method.4)
DEBUG:numba.core.ssa:on stmt: $phi16.0 = $14get_iter.5
DEBUG:numba.core.ssa:on stmt: jump

DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$pha_bins0_32.0'}), (4, {'res': '$e1_24.1'}), (6, {'lhs': '$pha_bins0_32.0', 'rhs': '$e1_24.1', 'res': '$6compare_op.2'}), (8, {'res': '$pha_bins1_38.3'}), (10, {'res': '$e1_210.4'}), (12, {'lhs': '$pha_bins1_38.3', 'rhs': '$e1_210.4', 'res': '$12compare_op.5'}), (14, {'lhs': '$6compare_op.2', 'rhs': '$12compare_op.5', 'res': '$14binary_and.6'}), (16, {'retval': '$14binary_and.6', 'castval': '$16return_value.7'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    e1_2 = arg(0, name=e1_2)                 ['e1_2']
    pha_bins0_3 =

initing ResponseDPI, with fname
/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/resp_tabs_ebins/drm_theta_36.0_phi_15.0_.fits
initing ResponseDPI, with fname
/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/resp_tabs_ebins/drm_theta_45.0_phi_0.0_.fits
initing ResponseDPI, with fname
/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/resp_tabs_ebins/drm_theta_45.0_phi_15.0_.fits
CPU times: user 10.5 s, sys: 2.96 s, total: 13.5 s
Wall time: 13.4 s


In [13]:
from config import solid_angle_dpi_fname
# or give the path directly
# solid_angle_dpi_fname = '/path/to/solid_angle_dpi.npy'
bkg_fname = os.path.join(work_dir,'bkg_estimation.csv')
solid_ang_dpi = np.load(solid_angle_dpi_fname)
bkg_df, bkg_name, PSnames, bkg_mod, ps_mods = parse_bkg_csv(bkg_fname, solid_ang_dpi,\
                    ebins0, ebins1, bl_dmask, rt_dir)
bkg_mod.has_deriv = False
bkg_mod_list = [bkg_mod]
Nsrcs = len(ps_mods)
if Nsrcs > 0:
    bkg_mod_list += ps_mods
    for ps_mod in ps_mods:
        ps_mod.has_deriv = False
    bkg_mod = CompoundModel(bkg_mod_list)
tmid = trigger_time
bkg_row = bkg_df.iloc[np.argmin(np.abs(tmid - bkg_df['time']))]
bkg_params = {pname:bkg_row[pname] for pname in\
            bkg_mod.param_names}
bkg_name = bkg_mod.name
bkg_params

['4U 1700-377', 'GRO J1655-40', 'GX 339-4', 'Sco X-1']
INFO:root:0.02
INFO:root:0.002
DEBUG:root:opening file: fwd_ray_trace_-0.10000_-0.48000_-0.08000_-0.46000_.npy
DEBUG:root:nbytes_total=95789408.0
DEBUG:root:opening file: fwd_ray_trace_-0.12000_-0.54000_-0.10000_-0.52000_.npy
DEBUG:root:nbytes_total=191578816.0
DEBUG:root:opening file: fwd_ray_trace_0.00000_-0.70000_0.02000_-0.68000_.npy
DEBUG:root:nbytes_total=287368224.0
DEBUG:root:opening file: fwd_ray_trace_-0.50000_-0.22000_-0.48000_-0.20000_.npy
DEBUG:root:nbytes_total=383157632.0


{'Background_bkg_rate_0': 0.0913703220701183,
 'Background_bkg_rate_1': 0.0661578002239374,
 'Background_bkg_rate_2': 0.0400898569026105,
 'Background_bkg_rate_3': 0.0394919934207801,
 'Background_bkg_rate_4': 0.0346780266835281,
 'Background_bkg_rate_5': 0.0351076584048365,
 'Background_bkg_rate_6': 0.033602492003955,
 'Background_bkg_rate_7': 0.0248610826847874,
 'Background_bkg_rate_8': 0.0176399476145076,
 'Background_flat_0': 0.0,
 'Background_flat_1': 0.0,
 'Background_flat_2': 0.1859632649428977,
 'Background_flat_3': 0.0766344244533236,
 'Background_flat_4': 0.3020855890115875,
 'Background_flat_5': 0.7611510180823338,
 'Background_flat_6': 0.8091430254293075,
 'Background_flat_7': 1.0,
 'Background_flat_8': 1.0,
 '4U 1700-377_imx': -0.0981485305770971,
 '4U 1700-377_imy': -0.4742076074486664,
 '4U 1700-377_rate_0': 0.0124901359218975,
 '4U 1700-377_rate_1': 0.0072425264886238,
 '4U 1700-377_rate_2': 0.0045295645864628,
 '4U 1700-377_rate_3': 0.0016876913514016,
 '4U 1700-377_r

In [14]:
pars_ = {}
pars_['Signal_theta'] = theta
pars_['Signal_phi'] = phi
for pname,val in list(bkg_params.items()):
    pars_[bkg_name+'_'+pname] = val
for pname,val in list(flux_params.items()):
    pars_['Signal_'+pname] = val
pars_

{'Signal_theta': 38.54132137017975,
 'Signal_phi': 137.65241966813443,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_0': 0.0913703220701183,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_1': 0.0661578002239374,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_2': 0.0400898569026105,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_3': 0.0394919934207801,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_4': 0.0346780266835281,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_5': 0.0351076584048365,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_6': 0.033602492003955,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_7': 0.0248610826847874,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_8': 0.0176399476145076,
 'Background+4U 1700-377+GRO J1655-40+G

In [15]:
comp_mod = CompoundModel([bkg_mod, sig_mod])

sig_miner = NLLH_ScipyMinimize_Wjacob('')

sig_llh_obj = LLH_webins(ev_data0, ebins0, ebins1, bl_dmask, has_err=True)

sig_llh_obj.set_model(comp_mod)

sig_miner.set_llh(sig_llh_obj)

In [16]:
fixed_pnames = list(pars_.keys())
fixed_vals = list(pars_.values())
trans = [None for i in range(len(fixed_pnames))]
sig_miner.set_trans(fixed_pnames, trans)
sig_miner.set_fixed_params(fixed_pnames, values=fixed_vals)
sig_miner.set_fixed_params(['Signal_A'], fixed=False)

In [17]:
%%time
flux_params['gamma'] = 0.8
flux_params['Epeak'] = 350.0
sig_mod.set_flux_params(flux_params)

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1110)
           2	LOAD_FAST(arg=0, lineno=1110)
           4	LOAD_ATTR(arg=0, lineno=1110)
           6	UNPACK_SEQUENCE(arg=3, lineno=1110)
           8	STORE_FAST(arg=2, lineno=1110)
          10	STORE_FAST(arg=3, lineno=1110)
          12	STORE_FAST(arg=4, lineno=1110)
          14	LOAD_GLOBAL(arg=1, lineno=1111)
          16	LOAD_METHOD(arg=2, lineno=1111)
          18	LOAD_FAST(arg=2, lineno=1111)
          20	LOAD_FAST(arg=4, lineno=1111)
          22	BUILD_TUPLE(arg=2, lineno=1111)
          24	CALL_METHOD(arg=1, lineno=1111)
          26	STORE_FAST(arg=5, lineno=1111)
          28	LOAD_GLOBAL(arg=3, lineno=1113)
          30	LOAD_FAST(arg=2, lineno=1113)
          32	CALL_FUNCTION(arg=1, lineno=1113)
          34	GET_ITER(arg=None, lineno=1113)
>         36	FOR_ITER(arg=70, lineno=1113)
          38	STORE_FAST(arg=6, lineno=1113)
          40	LOAD_GLOBAL(arg=3, lineno=1114)
          42	LOAD_FAST(arg=4, 

DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$phi50.2']
DEBUG:numba.core.byteflow:dispatch pc=52, inst=LOAD_GLOBAL(arg=3, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1']
DEBUG:numba.core.byteflow:dispatch pc=54, inst=LOAD_FAST(arg=3, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$52load_global.3']
DEBUG:numba.core.byteflow:dispatch pc=56, inst=CALL_FUNCTION(arg=1, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$52load_global.3', '$NphotonEs54.4']
DEBUG:numba.core.byteflow:dispatch pc=58, inst=GET_ITER(arg=None, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$56call_function.5']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=60, stack=('$phi50.0', '$phi50.1', '$58get_iter.6'), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=36 nstack_initial=1), State(pc_initial=60 nstack_initial=3)])
DEBUG:numba.core.byteflow:pending: deque([State(pc_initi

DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>,
            {'$phi104.0': {('$phi50.0', State(pc_initial=50 nstack_initial=3)),
                           ('$phi62.0', State(pc_initial=62 nstack_initial=4))},
             '$phi104.1': {('$phi50.1', State(pc_initial=50 nstack_initial=3)),
                           ('$phi62.1', State(pc_initial=62 nstack_initial=4))},
             '$phi106.0': {('$phi104.0',
                            State(pc_initial=104 nstack_initial=2)),
                           ('$phi38.0', State(pc_initial=38 nstack_initial=2))},
             '$phi36.0': {('$34get_iter.15',
                           State(pc_initial=0 nstack_initial=0)),
                          ('$phi104.0', State(pc_initial=104 nstack_initial=2)),
                          ('$phi38.0', State(pc_initial=38 nstack_initial=2))},
             '$phi38.0': {('$34get_iter.15',
                           State(pc_initial=0 nstack_initial=0)),
                          ('$phi1

DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$resp_dpi2.0'}), (4, {'item': '$resp_dpi2.0', 'res': '$4load_attr.1'}), (6, {'iterable': '$4load_attr.1', 'stores': ['$6unpack_sequence.2', '$6unpack_sequence.3', '$6unpack_sequence.4'], 'tupleobj': '$6unpack_sequence.5'}), (8, {'value': '$6unpack_sequence.2'}), (10, {'value': '$6unpack_sequence.3'}), (12, {'value': '$6unpack_sequence.4'}), (14, {'res': '$14load_global.6'}), (16, {'item': '$14load_global.6', 'res': '$16load_method.7'}), (18, {'res': '$ndets18.8'}), (20, {'res': '$Nphabins20.9'}), (22, {'items': ['$ndets18.8', '$Nphabins20.9'], 'res': '$22build_tuple.10'}), (24, {'func': '$16load_method.7', 'args': ['$22build_tuple.10'], 'res': '$24call_method.11'}), (26, {'value': '$24call_method.11'}), (28, {'res': '$28load_global.12'}), (30, {'res': '$ndets30.13'}), (32, {'func': 

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x7fc1803bbbb0>
DEBUG:numba.core.ssa:on stmt: resp_dpi = arg(0, name=resp_dpi)
DEBUG:numba.core.ssa:on stmt: photon_fluxes = arg(1, name=photon_fluxes)
DEBUG:numba.core.ssa:on stmt: $4load_attr.1 = getattr(value=resp_dpi, attr=shape)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.5 = exhaust_iter(value=$4load_attr.1, count=3)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.2 = static_getitem(value=$6unpack_sequence.5, index=0, index_var=None, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.3 = static_getitem(value=$6unpack_sequence.5, index=1, index_var=None, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.4 = static_getitem(value=$6unpack_sequence.5, index=2, index_var=None, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: ndets = $6unpack_sequence.2
DEBUG:numba.core.ssa:on stmt: N

DEBUG:numba.core.ssa:SSA violators set()
CPU times: user 342 ms, sys: 45.8 ms, total: 388 ms
Wall time: 343 ms


In [18]:
t0 = trigger_time - 0.512
t1 = t0 + 2.048
sig_llh_obj.set_time(t0, t1)

In [19]:
%%time
pars, nllh, res = sig_miner.minimize()

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=283)
           2	LOAD_CONST(arg=1, lineno=283)
           4	STORE_FAST(arg=4, lineno=283)
           6	LOAD_GLOBAL(arg=0, lineno=284)
           8	LOAD_FAST(arg=3, lineno=284)
          10	CALL_FUNCTION(arg=1, lineno=284)
          12	GET_ITER(arg=None, lineno=284)
>         14	FOR_ITER(arg=354, lineno=284)
          18	STORE_FAST(arg=5, lineno=284)
          20	LOAD_FAST(arg=2, lineno=285)
          22	LOAD_FAST(arg=5, lineno=285)
          24	BINARY_SUBSCR(arg=None, lineno=285)
          26	LOAD_CONST(arg=2, lineno=285)
          28	COMPARE_OP(arg=2, lineno=285)
          30	POP_JUMP_IF_FALSE(arg=60, lineno=285)
          32	LOAD_GLOBAL(arg=1, lineno=286)
          34	LOAD_METHOD(arg=2, lineno=286)
          36	LOAD_GLOBAL(arg=3, lineno=286)
          38	LOAD_FAST(arg=0, lineno=286)
          40	LOAD_FAST(arg=5, lineno=286)
          42	BINARY_SUBSCR(arg=None, lineno=286)
          44	LOAD_FAST(arg=1, lineno=

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=283)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_CONST(arg=1, lineno=283)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=STORE_FAST(arg=4, lineno=283)
DEBUG:numba.core.byteflow:stack ['$const2.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=LOAD_GLOBAL(arg=0, lineno=284)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=8, inst=LOAD_FAST(arg=3, lineno=284)
DEBUG:numba.core.byteflow:stack ['$6load_global.1']
DEBUG:numba.core.byteflow:dispatch pc=10, inst=CALL_FUNCTION(arg=1, lineno=284)
DEBUG:numba.core.byteflow:stack ['$6load_global.1', '$size8.2']
DEBUG:numba.core.byteflow:dispatch pc=12, inst=GET_ITER(arg=None, lineno=284)
DEBUG:numba.core.byteflow:stack ['$10call_function.3']
DEBUG:numba.core.byteflow:e

DEBUG:numba.core.byteflow:stack ['$phi72.0', '$72load_global.1']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=LOAD_GLOBAL(arg=4, lineno=288)
DEBUG:numba.core.byteflow:stack ['$phi72.0', '$74load_method.2']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=0, lineno=288)
DEBUG:numba.core.byteflow:stack ['$phi72.0', '$74load_method.2', '$76load_global.3']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_FAST(arg=5, lineno=288)
DEBUG:numba.core.byteflow:stack ['$phi72.0', '$74load_method.2', '$76load_global.3', '$mu78.4']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=BINARY_SUBSCR(arg=None, lineno=288)
DEBUG:numba.core.byteflow:stack ['$phi72.0', '$74load_method.2', '$76load_global.3', '$mu78.4', '$i80.5']
DEBUG:numba.core.byteflow:dispatch pc=84, inst=LOAD_FAST(arg=1, lineno=288)
DEBUG:numba.core.byteflow:stack ['$phi72.0', '$74load_method.2', '$76load_global.3', '$82binary_subscr.6']
DEBUG:numba.core.byteflow:dispatch pc=86, inst=LOAD_FAST(arg=5, lineno=288)
DEBUG:numba

DEBUG:numba.core.byteflow:stack ['$phi150.0', '$152load_method.2', '$154load_global.3']
DEBUG:numba.core.byteflow:dispatch pc=158, inst=LOAD_FAST(arg=5, lineno=292)
DEBUG:numba.core.byteflow:stack ['$phi150.0', '$152load_method.2', '$154load_global.3', '$mu156.4']
DEBUG:numba.core.byteflow:dispatch pc=160, inst=BINARY_SUBSCR(arg=None, lineno=292)
DEBUG:numba.core.byteflow:stack ['$phi150.0', '$152load_method.2', '$154load_global.3', '$mu156.4', '$i158.5']
DEBUG:numba.core.byteflow:dispatch pc=162, inst=LOAD_FAST(arg=1, lineno=292)
DEBUG:numba.core.byteflow:stack ['$phi150.0', '$152load_method.2', '$154load_global.3', '$160binary_subscr.6']
DEBUG:numba.core.byteflow:dispatch pc=164, inst=LOAD_FAST(arg=5, lineno=292)
DEBUG:numba.core.byteflow:stack ['$phi150.0', '$152load_method.2', '$154load_global.3', '$160binary_subscr.6', '$sig162.7']
DEBUG:numba.core.byteflow:dispatch pc=166, inst=BINARY_SUBSCR(arg=None, lineno=292)
DEBUG:numba.core.byteflow:stack ['$phi150.0', '$152load_method.2', 

DEBUG:numba.core.byteflow:stack ['$phi226.0', '$228load_method.2', '$230load_global.3', '$236binary_subscr.6']
DEBUG:numba.core.byteflow:dispatch pc=240, inst=LOAD_FAST(arg=5, lineno=296)
DEBUG:numba.core.byteflow:stack ['$phi226.0', '$228load_method.2', '$230load_global.3', '$236binary_subscr.6', '$sig238.7']
DEBUG:numba.core.byteflow:dispatch pc=242, inst=BINARY_SUBSCR(arg=None, lineno=296)
DEBUG:numba.core.byteflow:stack ['$phi226.0', '$228load_method.2', '$230load_global.3', '$236binary_subscr.6', '$sig238.7', '$i240.8']
DEBUG:numba.core.byteflow:dispatch pc=244, inst=CALL_FUNCTION(arg=2, lineno=296)
DEBUG:numba.core.byteflow:stack ['$phi226.0', '$228load_method.2', '$230load_global.3', '$236binary_subscr.6', '$242binary_subscr.9']
DEBUG:numba.core.byteflow:dispatch pc=246, inst=CALL_METHOD(arg=1, lineno=296)
DEBUG:numba.core.byteflow:stack ['$phi226.0', '$228load_method.2', '$244call_function.10']
DEBUG:numba.core.byteflow:dispatch pc=248, inst=STORE_FAST(arg=6, lineno=296)
DEBUG:

...

DEBUG:numba.core.ssa:on stmt: $30binary_multiply.11 = $const24.8 * $28load_attr.10
DEBUG:numba.core.ssa:on stmt: $const34.13 = const(int, 2)
DEBUG:numba.core.ssa:on stmt: $36binary_power.14 = sig ** $const34.13
DEBUG:numba.core.ssa:on stmt: $38binary_multiply.15 = $30binary_multiply.11 * $36binary_power.14
DEBUG:numba.core.ssa:on stmt: $40binary_true_divide.16 = $const22.7 / $38binary_multiply.15
DEBUG:numba.core.ssa:on stmt: $const42.17 = const(float, 0.5)
DEBUG:numba.core.ssa:on stmt: norm_A = $40binary_true_divide.16 ** $const42.17
DEBUG:numba.core.ssa:on stmt: $48load_global.19 = global(num_factorial: CPUDispatcher(<function num_factorial at 0x7fc1617483a0>))
DEBUG:numba.core.ssa:on stmt: fact = call $48load_global.19(N, func=$48load_global.19, args=[Var(N, LLH.py:265)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $62binary_multiply.25 = dmu * Nmu
DEBUG:numba.core.ssa:on stmt: $const64.26 = const(int, 2)
DEBUG:numba.core.ssa:on stmt: $66binary_true_divide.27 = $62binary_mult

DEBUG:numba.core.ssa:on stmt: $66binary_true_divide.27 = $62binary_multiply.25 / $const64.26
DEBUG:numba.core.ssa:on stmt: mu0 = mu - $66binary_true_divide.27
DEBUG:numba.core.ssa:on stmt: $const74.30 = const(int, 0)
DEBUG:numba.core.ssa:on stmt: $76compare_op.31 = mu0 < $const74.30
DEBUG:numba.core.ssa:find_def var='mu0' stmt=$76compare_op.31 = mu0 < $const74.30
DEBUG:numba.core.ssa:on stmt: bool78 = global(bool: <class 'bool'>)
DEBUG:numba.core.ssa:on stmt: $78pred = call bool78($76compare_op.31, func=bool78, args=(Var($76compare_op.31, LLH.py:271),), kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: branch $78pred, 80, 84
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 80
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x7fc180a1e310>
DEBUG:numba.core.ssa:on stmt: mu0.1 = const(float, 0.0)
DEBUG:numba.core.ssa:on stmt: jump 84
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 84
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x7fc180a1e3

DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=20, stack=('$18get_iter.7',), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=20 nstack_initial=1)])
DEBUG:numba.core.byteflow:stack: ['$phi20.0']
DEBUG:numba.core.byteflow:dispatch pc=20, inst=FOR_ITER(arg=12, lineno=258)
DEBUG:numba.core.byteflow:stack ['$phi20.0']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=34, stack=(), blockstack=(), npush=0), Edge(pc=22, stack=('$phi20.0', '$20for_iter.2'), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=34 nstack_initial=0), State(pc_initial=22 nstack_initial=2)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=34, inst=LOAD_FAST(arg=1, lineno=260)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=36, inst=RETURN_VALUE(arg=None, lineno=260)
DEBUG:numba.core.byteflow:stack ['$res34.0']
DEBUG:numba.core.byteflow:end state. edges=[]
DEBUG:numba.core.byteflow:pending: deq

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x7fc120241cd0>
DEBUG:numba.core.ssa:on stmt: N = arg(0, name=N)
DEBUG:numba.core.ssa:on stmt: res = const(float, 1.0)
DEBUG:numba.core.ssa:on stmt: $6load_global.1 = global(range: <class 'range'>)
DEBUG:numba.core.ssa:on stmt: $const8.2 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $const12.4 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $14binary_add.5 = N + $const12.4
DEBUG:numba.core.ssa:on stmt: $16call_function.6 = call $6load_global.1($const8.2, $14binary_add.5, func=$6load_global.1, args=[Var($const8.2, LLH.py:258), Var($14binary_add.5, LLH.py:258)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $18get_iter.7 = getiter(value=$16call_function.6)
DEBUG:numba.core.ssa:on stmt: $phi20.0 = $18get_iter.7
DEBUG:numba.core.ssa:on stmt: jump 20
DEBUG:numba.core.ssa:==== SSA block analysis pass on 20
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefs

DEBUG:numba.core.ssa:insert phi node res.2 = phi(incoming_values=[], incoming_blocks=[]) at 20
DEBUG:numba.core.ssa:find_def_from_bottom label 0
DEBUG:numba.core.ssa:incoming_def res = const(float, 1.0)
DEBUG:numba.core.ssa:find_def_from_bottom label 22
DEBUG:numba.core.ssa:incoming_def res.1 = $28inplace_multiply.4
DEBUG:numba.core.ssa:replaced with: $28inplace_multiply.4 = inplace_binop(fn=<built-in function imul>, immutable_fn=<built-in function mul>, lhs=res.2, rhs=i, static_lhs=Undefined, static_rhs=Undefined)
DEBUG:numba.core.ssa:on stmt: res.1 = $28inplace_multiply.4
DEBUG:numba.core.ssa:on stmt: jump 20
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 34
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x7fc120244af0>
DEBUG:numba.core.ssa:on stmt: $36return_value.1 = cast(value=res)
DEBUG:numba.core.ssa:find_def var='res' stmt=$36return_value.1 = cast(value=res)
DEBUG:numba.core.ssa:find_def_from_top label 34
DEBUG:numba.core.ssa:idom 20 from label 34
DEBUG

In [20]:
print(res)
print(nllh)
print(pars)

[      fun: 45893.461149418945
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00255971])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 8
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([0.00692558])]
[45893.461149418945]
[[0.006925579106911662]]


In [22]:
%%time
pars_['Signal_A'] = 1e-10
bkg_nllh = -sig_llh_obj.get_logprob(pars_)
print (bkg_nllh)
sqrtTS = np.sqrt(2.*(bkg_nllh - nllh[0]))
print (sqrtTS)

46038.74332721739
17.045948363082807
CPU times: user 6.62 ms, sys: 2.07 ms, total: 8.69 ms
Wall time: 7.17 ms
